In [1]:
from sklearn import datasets
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np

In [5]:
data = load_breast_cancer()

x = data.data
y = data.target
features = data.feature_names

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=3)

# this scales the data so it is easier to work with, in this case to prevent the data from going to undefined since the numbers can get very small without scaling it
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [6]:
# Feedforward Neural Network
class FNN():
  # this method is a constructor and is used to create an instance of the feedforward neural network model containing the training data
  # parameters
  # learning_rate: this contains a float which is used to change how big of a jump the data will make
  # epoch: this contains a integer which is used to repeat the training epoch times
  # x: this contains the training dataset examples
  # y: this contains the training dataset answers
  def __init__(self, learning_rate, epoch, x, y):
    self.learning_rate = learning_rate
    self.epoch = epoch
    self.x = x
    self.y = y
    self.number_data, self.number_feature = x.shape
    self.input_weights = np.zeros((self.number_feature, 4))
    self.bias = 1
    self.bias_output = 1
    self.hidden_weights = np.zeros(4)

  # this method is used for back propagation
  # parameters
  # hidden_input: this contains an array that contains the input values combined with their weights
  # hidden_output: this contains an array that contains the hidden_input values but with relu applied to it
  # output_input: this contains an array that contains the hidden values combined with their weights
  # output_output: this contains an array that contains the output_input values but with sigmoid applied to it
  def backward_pass(self, hidden_input, hidden_output, output_input, output_output):
    # Calculate errors
    output_error = output_output - self.y
    hidden_error = np.outer(output_error, self.hidden_weights)

    # Calculating gradients
    output_weights_gradient = (1/self.number_data) * np.dot(hidden_output.T, output_error)
    output_grad_bias = (1/self.number_data) * np.sum(output_error)
    hidden_weights_gradient = (1/self.number_data) * np.dot(self.x.T, hidden_error)
    hidden_grad_bias = (1/self.number_data) * np.sum(hidden_error)

    # Updating weights and biases
    self.hidden_weights -= self.learning_rate * output_weights_gradient
    self.bias_output -= self.learning_rate * output_grad_bias
    self.input_weights -= self.learning_rate * hidden_weights_gradient
    self.bias -= self.learning_rate * hidden_grad_bias

  # trains the model
  def fit(self):
    for p in range(self.epoch):
      # input to hidden
      hidden_input = np.dot(self.x, self.input_weights) + self.bias
      relu_activation = np.maximum(0, hidden_input)

      # hidden to output using sigmoid since it is a binary classification
      output_hidden = np.dot(relu_activation, self.hidden_weights) + self.bias_output
      Sigmoid = 1 / (1 + np.exp(-output_hidden))

      # gradiants
      self.backward_pass(hidden_input, relu_activation, output_hidden, Sigmoid)

  # finds the accuracy with the testing dataset
  # parameters
  # x_test: contains the testing dataset examples
  # y_test: contains the testing dataset answers
  def accuracy(self, x_test, y_test):
    # calculate the outputs
    hidden_input = np.dot(x_test, self.input_weights) + self.bias
    relu_activation = np.maximum(0, hidden_input)

    # hidden to output
    output_hidden = np.dot(relu_activation, self.hidden_weights) + self.bias_output
    Sigmoid = 1 / (1 + np.exp(-output_hidden))

    accuracy = 0
    # finds accuracy
    for i in range(len(Sigmoid)):
      # uses the threshold to determine if it is 1 or 0
      if Sigmoid[i] > .5:
        # checks if the guess was correct, if it was increment by 1
        if y_test[i] == 1:
          accuracy += 1
      else:
        # checks if the guess was correct, if it was increment by 1
        if y_test[i] == 0:
          accuracy += 1

    # returns the percentage
    return (accuracy / len(y_test)) * 100

In [7]:
# Feedforward Neural Networks running
print("-----------------------------------FNN-----------------------------------")
Feedforward_Neural_Network = FNN(0.1, 100, X_train_scaled, y_train)
Feedforward_Neural_Network.fit()
print("Accuracy:",Feedforward_Neural_Network.accuracy(X_test_scaled, y_test))

-----------------------------------FNN-----------------------------------
Accuracy: 95.32163742690058
